## Setup

In [ ]:
import pandas as pd
import requests
import json
import math
import os
from dotenv import load_dotenv

load_dotenv()

NOTION_SECRET = os.getenv("NOTION_SECRET")
DATABASE_ID = os.getenv("DATABASE_ID")

## Get data from google

In [ ]:
def google_book_search(title, author, publisher):
    search_terms = " ".join(filter(None, [title, author, publisher]))
    url = 'https://www.googleapis.com/books/v1/volumes?q='
    response = requests.get(url+search_terms)
    data = response.json()
    # Normalizing data
    df = pd.json_normalize(data, record_path=['items'])
    return df


search_terms = "maniac labatut"
url = 'https://www.googleapis.com/books/v1/volumes?q='
response = requests.get(url+search_terms)
data = response.json()
# # Normalizing data
df = pd.json_normalize(data, record_path=['items'])

In [ ]:
def query_databases(secret_key, database_id):
    url = "https://api.notion.com/v1/databases/"+database_id+'/query'

    payload = {'id': database_id}
    headers = {
        'Notion-Version': '2021-05-13',
        'Authorization': 'Bearer '+secret_key
    }

    response = requests.request(
        "POST", url, headers=headers, data=payload)
    print(f"The response code is {response.status_code}")
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    else:
        return response.json()

In [ ]:
res = query_databases(NOTION_SECRET, DATABASE_ID)

In [ ]:
res.get('results')[0].get('properties')[
    'Summary']['rich_text'][0].get('plain_text')

In [ ]:
notion_columns = ['Category', 'Publisher', 'Summary', 'Current page', 'Link',
                  'Total pages', 'Date started', 'Author', 'Title', 'url', 'page_id']
notion = pd.DataFrame(columns=notion_columns)
print(notion.columns)
print(notion.head())
for page in res.get('results'):
    properties = page.get('properties')
    try:
        author = properties.get('Author').get('rich_text')[0].get('plain_text')
    except IndexError:
        author = None
    try:
        title = properties.get('Title').get('title')[0].get('plain_text')
    except IndexError:
        title = None
    try:
        publisher = properties['Publisher']['select']['name']
    except KeyError:
        publisher = None
    try:
        category = properties['Category']['select']['name']
    except KeyError:
        category = None
    try:
        summary = properties['Summary']['rich_text'][0]['plain_text']
    except IndexError:
        summary = None
    try:
        current_page = properties['Current page']['number']
    except KeyError:
        current_page = None
    try:
        link = properties['Link']['url']
    except KeyError:
        link = None
    total_pages = properties['Total pages']['number']
    try:
        date_started = properties['Date started']['date']['start']
    except KeyError:
        date_started = None

    url = page.get('url')
    page_id = url[-32:]
    # concat the data
    notion = pd.concat([notion, pd.DataFrame([[category, publisher, summary, current_page, link, total_pages,
                       date_started, author, title, url, page_id]], columns=notion_columns)], ignore_index=True)
# drop rows without title
notion = notion.dropna(subset=['Title'])

In [ ]:
google_results = pd.DataFrame()

for book in notion.itertuples():

    google_results = pd.concat([google_results, google_book_search(
        book.Title, book.Author, book.Publisher)], ignore_index=True)

In [ ]:

google_data = google_results[['selfLink', 'volumeInfo.title',
                              'volumeInfo.subtitle', 'volumeInfo.authors', 'volumeInfo.publisher',
                              'volumeInfo.publishedDate', 'volumeInfo.description', 'volumeInfo.pageCount', 'volumeInfo.categories',
                              'volumeInfo.imageLinks.smallThumbnail', 'volumeInfo.imageLinks.thumbnail', 'saleInfo.country', 'saleInfo.retailPrice.amount',
                              'saleInfo.retailPrice.currencyCode'
                              ]]

In [ ]:
import pandas as pd


def clean_google_data(google_df, notion_df):
    filtered_results = []

    for _, notion_row in notion_df.iterrows():
        try:
            # Convert Notion title to lowercase and split into words
            notion_title_words = notion_row['Title'].lower().split()
            # Create a regex pattern to match all words
            pattern = '.*'.join(notion_title_words)
            # Filter google_df by title using regex
            matches = google_df[google_df['volumeInfo.title'].str.lower(
            ).str.contains(pattern, regex=True, na=False)]
            # add page_id to matches as column
            matches['page_id'] = notion_row['page_id']

        except TypeError as e:
            print(f"TypeError encountered while filtering by title: {e}")
            continue

        try:
            # Further filter by author if available
            if pd.notna(notion_row.get('Author')):
                tmp_df = matches[matches['volumeInfo.authors'].apply(
                    lambda authors: notion_row['Author'] in authors if isinstance(authors, list) else False)]
                if not tmp_df.empty:  # If there are matches, keep them
                    matches = tmp_df
        except TypeError as e:
            print(f"TypeError encountered while filtering by author: {e}")
            continue

        try:
            # Further filter by publisher if available
            if pd.notna(notion_row.get('Publisher')):
                tmp_df = matches[matches['volumeInfo.publisher'].apply(
                    lambda publisher: notion_row['Publisher'] == publisher if isinstance(publisher, str) else False)]
                if not tmp_df.empty:  # If there are matches, keep them
                    matches = tmp_df

        except TypeError as e:
            print(f"TypeError encountered while filtering by publisher: {e}")
            continue
        try:
            # If there are matches, keep the latest by published_date
            latest_match = matches.sort_values(
                by='volumeInfo.publishedDate', ascending=False).iloc[0]
            filtered_results.append(latest_match)
        except IndexError as e:
            # only single match, append
            if not matches.empty:
                filtered_results.append(matches)
            else:
                print(f"No match found for {notion_row['Title']}")
            continue
        except KeyError:
            if not matches.empty:
                filtered_results.append(matches)
            else:
                print(f"No match found for {notion_row['Title']}")
            continue

    # Convert the list of filtered results to a DataFrame
    filtered_df = pd.DataFrame(filtered_results)
    return filtered_df

In [ ]:
clean_google_data = clean_google_data(google_data, notion)
clean_google_data

In [ ]:
clean_google_data

In [ ]:
# merge clean google and notion data on page_id
complete = pd.merge(notion, clean_google_data, on='page_id', how='left')
complete

## Update Notion with Google Data

In [ ]:
# Update a property on a page based on property type
def update_page(row, property_name, property_type, data_column, verbose=False):
    url = f"https://api.notion.com/v1/pages/{row.page_id}"

    # erxtract the property value
    property_value = row[data_column]

    # Check the property type and create the payload
    if property_type == 'date':
        property_payload = {
            "start": property_value
        }
    elif property_type == 'url':
        property_payload = property_value
    elif property_type == 'number':
        property_payload = property_value
    elif property_type == 'rich_text':
        property_payload = [{
            "type": "text",
            "text": {
                "content": property_value
            }
        }]
    elif property_type == 'select':
        property_payload = {
            "name": property_value
        }

    payload = json.dumps({
        "properties": {
            property_name: {
                property_type: property_payload
            }
        }
    })

    headers = {
        'Content-Type': 'application/json',
        'Notion-Version': '2021-05-13',
        'Authorization': f'Bearer {NOTION_SECRET}'
    }

    response = requests.request(
        "PATCH", url, headers=headers, data=payload)
    if verbose:
        print(response.status_code)
    errors = []
    if response.status_code != 200:
        errors.append(response.text)
    return errors

### update publishing dates

In [ ]:
clean_google_data.apply(lambda row: update_page(
    row, "Published", "date", "volumeInfo.publishedDate"), axis=1)

In [ ]:
clean_google_data.apply(lambda row: update_page(
    row, "Link", "url", "selfLink"), axis=1)

In [ ]:
clean_google_data.apply(lambda row: update_page(
    row, "Publisher", "select", "volumeInfo.publisher"), axis=1)

In [ ]:
clean_google_data.apply(lambda row: update_page(
    row, "Total pages", "number", "volumeInfo.pageCount"), axis=1)

In [ ]:
summary_errors = clean_google_data.apply(lambda row: update_page(
    row, "Summary", "rich_text", "volumeInfo.description"), axis=1)

In [ ]:
def update_page_icon(row, data_column, icon_or_cover):

    page_id = row['page_id']
    property_value = row[data_column]

    url = f"https://api.notion.com/v1/pages/{page_id}"

    payload = json.dumps({icon_or_cover: {
        "type": "external",
        "external": {
                "url": property_value
        }}})
    headers = {
        'Content-Type': 'application/json',
        'Notion-Version': '2021-05-13',
        'Authorization': f'Bearer {NOTION_SECRET}'
    }

    response = requests.request(
        "PATCH", url, headers=headers, data=payload)
    errors = []
    if response.status_code != 200:
        print(payload)
        errors.append(response.text)
    return errors

In [ ]:
icon_errors = clean_google_data.apply(lambda row: update_page_icon(
    row, 'volumeInfo.imageLinks.smallThumbnail', 'icon'), axis=1)

In [ ]:
cover_errors = clean_google_data.apply(lambda row: update_page_icon(
    row, 'volumeInfo.imageLinks.smallThumbnail', 'cover'), axis=1)